In [1]:
import pandas as pd
import Levenshtein
import itertools
from fuzzywuzzy import fuzz
import ast
from selenium import webdriver
import time
import webbrowser

In [2]:
recipe_df = pd.read_csv('../data/processed/recipe_df.csv')

recipe_df.head()

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),calories(kcal),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,tipo_plato,main_category,category,num_ingredients
0,https://content-recetas.lecturas.com/medio/201...,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nico...,"['patatas pequeñas rojas', 'huevos', 'judías v...",60.0,252.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,12
1,https://content-recetas.lecturas.com/medio/201...,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-peri...,"['foie micuit', 'beicon gruesas', 'champiñones...",25.0,386.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,9
2,https://content-recetas.lecturas.com/medio/202...,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-ro...,"['bacalao desalado', 'cebolleta', 'escarolas',...",35.0,NaN,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,16
3,https://content-recetas.lecturas.com/medio/201...,Ensalada de ceviche de langostinos y cebolla,https://www.lecturas.com/recetas/ensalada-cevi...,"['langostinos', 'pimienta', 'limas', 'tomates'...",25.0,176.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,12
4,https://content-recetas.lecturas.com/medio/201...,"Ensalada de pollo, remolacha y manzana",https://www.lecturas.com/recetas/ensalada-poll...,"['pechugas de pollo', 'manzanas ácida', 'col l...",40.0,203.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,16


In [3]:
#recipe_df["ingredient"].value_counts()

In [4]:
input_ing = input('¿Qué tienes hoy en la nevera?')

¿Qué tienes hoy en la nevera? pechuga de pollo, arroz, pimiento, aceite, sal, mantequilla, brécol, calabacin, leche


In [5]:
input_ing

'pechuga de pollo, arroz, pimiento, aceite, sal, mantequilla, brécol, calabacin, leche'

In [6]:
def input_list(string):
    input_list = []
    str_split = string.split(", ")
    for x in str_split:
        input_list.append(x)
    return input_list

input_ingred = input_list(input_ing)

In [7]:
def fuzzy_columns(column, input_string):
    ingred_list = ast.literal_eval(column)
    for ingred in ingred_list:
        dif = fuzz.token_set_ratio(column, input_string)
    if dif > 70:
        return True
    else:
        return False


In [8]:
for ingred in input_ingred:
    recipe_df[ingred] = recipe_df.apply(lambda x: fuzzy_columns(x["ingredients_list"], ingred), axis=1)

In [9]:
recipe_df.head()

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),calories(kcal),gluten_free,egg_free,sucrose_fructose_free,low_sodium,...,num_ingredients,pechuga de pollo,arroz,pimiento,aceite,sal,mantequilla,brécol,calabacin,leche
0,https://content-recetas.lecturas.com/medio/201...,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nico...,"['patatas pequeñas rojas', 'huevos', 'judías v...",60.0,252.0,0,0,0,0,...,12,False,False,False,True,True,False,False,False,False
1,https://content-recetas.lecturas.com/medio/201...,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-peri...,"['foie micuit', 'beicon gruesas', 'champiñones...",25.0,386.0,0,0,0,0,...,9,False,False,False,True,False,False,False,False,False
2,https://content-recetas.lecturas.com/medio/202...,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-ro...,"['bacalao desalado', 'cebolleta', 'escarolas',...",35.0,NaN,0,0,0,0,...,16,False,False,False,True,True,False,False,False,False
3,https://content-recetas.lecturas.com/medio/201...,Ensalada de ceviche de langostinos y cebolla,https://www.lecturas.com/recetas/ensalada-cevi...,"['langostinos', 'pimienta', 'limas', 'tomates'...",25.0,176.0,0,0,0,0,...,12,False,False,False,False,True,False,False,False,False
4,https://content-recetas.lecturas.com/medio/201...,"Ensalada de pollo, remolacha y manzana",https://www.lecturas.com/recetas/ensalada-poll...,"['pechugas de pollo', 'manzanas ácida', 'col l...",40.0,203.0,0,0,0,0,...,16,False,False,False,True,True,False,False,False,False


In [10]:
recipe_df["active_ingred"] = recipe_df.select_dtypes(include=['bool']).sum(axis=1)

In [11]:
def num_missing_ingred(recipe_ingred, active_ingred):
    missing_ingred = recipe_ingred - active_ingred
    return missing_ingred

recipe_df["num_missing_ingred"] = recipe_df.apply(lambda x: num_missing_ingred(x["num_ingredients"], x["active_ingred"]), axis=1)

In [12]:
recipe_df.head(3)

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),calories(kcal),gluten_free,egg_free,sucrose_fructose_free,low_sodium,...,arroz,pimiento,aceite,sal,mantequilla,brécol,calabacin,leche,active_ingred,num_missing_ingred
0,https://content-recetas.lecturas.com/medio/201...,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nico...,"['patatas pequeñas rojas', 'huevos', 'judías v...",60.0,252.0,0,0,0,0,...,False,False,True,True,False,False,False,False,2,10
1,https://content-recetas.lecturas.com/medio/201...,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-peri...,"['foie micuit', 'beicon gruesas', 'champiñones...",25.0,386.0,0,0,0,0,...,False,False,True,False,False,False,False,False,1,8
2,https://content-recetas.lecturas.com/medio/202...,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-ro...,"['bacalao desalado', 'cebolleta', 'escarolas',...",35.0,NaN,0,0,0,0,...,False,False,True,True,False,False,False,False,2,14


In [13]:
def rating(match, missing):
    result = 0.75*match-0.25*missing
    return result

recipe_df["recipe_rate"] = recipe_df.apply(lambda x: rating(x["active_ingred"], x["num_missing_ingred"]), axis=1)

In [14]:
recipe_df.head(3)

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),calories(kcal),gluten_free,egg_free,sucrose_fructose_free,low_sodium,...,pimiento,aceite,sal,mantequilla,brécol,calabacin,leche,active_ingred,num_missing_ingred,recipe_rate
0,https://content-recetas.lecturas.com/medio/201...,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nico...,"['patatas pequeñas rojas', 'huevos', 'judías v...",60.0,252.0,0,0,0,0,...,False,True,True,False,False,False,False,2,10,-1.00
1,https://content-recetas.lecturas.com/medio/201...,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-peri...,"['foie micuit', 'beicon gruesas', 'champiñones...",25.0,386.0,0,0,0,0,...,False,True,False,False,False,False,False,1,8,-1.25
2,https://content-recetas.lecturas.com/medio/202...,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-ro...,"['bacalao desalado', 'cebolleta', 'escarolas',...",35.0,NaN,0,0,0,0,...,False,True,True,False,False,False,False,2,14,-2.00


In [20]:
filter_principal = recipe_df['main_category'] == "principal"

output_df = recipe_df[filter_principal].sort_values(by=['recipe_rate', 'num_missing_ingred'], ascending=[False, True]).head(10)

output_df

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),calories(kcal),gluten_free,egg_free,sucrose_fructose_free,low_sodium,...,pimiento,aceite,sal,mantequilla,brécol,calabacin,leche,active_ingred,num_missing_ingred,recipe_rate
2235,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",Pechuga de pavo con chips de calabacín,https://www.lecturas.com/recetas/pechuga-pavo-...,"['pechuga de pavo', 'calabacín', 'espinacas fr...",40.0,197.0,1,0,0,0,...,True,True,True,False,False,False,False,4,3,2.25
1475,https://content-recetas.lecturas.com/medio/201...,Huevos encapotados,https://www.lecturas.com/recetas/huevos-encapo...,"['huevo', 'leche', 'harina', 'mantequilla', 'p...",35.0,630.0,0,0,0,0,...,True,True,True,True,False,False,True,5,6,2.25
281,https://content-recetas.lecturas.com/medio/202...,Crema de zanahorias asadas con comino y limón,https://www.lecturas.com/recetas/crema-zanahor...,"['zanahorias', 'diente de ajo', 'comino', 'lim...",40.0,NaN,0,0,0,0,...,False,True,True,False,False,False,True,4,4,2.00
775,https://content-recetas.lecturas.com/medio/201...,Timbal de verduras,https://www.lecturas.com/recetas/timbal-verdur...,"['berenjena', 'pimiento rojo', 'pimiento verde...",70.0,175.0,0,0,0,0,...,True,True,True,True,False,False,True,5,7,2.00
674,https://content-recetas.lecturas.com/medio/201...,Pimientos asados,https://www.lecturas.com/recetas/pimientos-asa...,"['pimiento rojo 1 kilo', 'diente de ajo', 'ace...",50.0,32.0,1,0,0,0,...,True,True,True,False,False,False,False,3,2,1.75
82,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",Ensalada de pollo con arroz y tomates,https://www.lecturas.com/recetas/ensalada-poll...,"['pechuga de pollo asada en dos mitades', 'arr...",15.0,337.0,0,0,0,0,...,False,True,True,False,False,False,False,4,5,1.75
368,https://content-recetas.lecturas.com/medio/201...,Puré de patata,https://www.lecturas.com/recetas/pure-patata_2...,"['patatas', 'mantequilla', 'leche', 'nata líqu...",50.0,NaN,0,0,0,0,...,False,True,True,True,False,False,True,4,5,1.75
970,https://content-recetas.lecturas.com/medio/201...,Sartenada de arroz con pimientos y tomatitos,https://www.lecturas.com/recetas/sartenada-arr...,"['arroz basmati', 'pimiento rojo', 'pimiento a...",20.0,310.0,0,0,0,0,...,True,True,True,False,False,False,False,4,5,1.75
998,https://content-recetas.lecturas.com/medio/201...,Risotto con almejas,https://www.lecturas.com/recetas/risotto-almej...,"['arroz', 'almeja', 'cebolleta', 'pimiento ama...",NaN,NaN,0,0,0,0,...,True,True,True,False,False,False,False,4,5,1.75
1051,https://content-recetas.lecturas.com/medio/201...,Arroz con sofrito de pimiento y panceta,https://www.lecturas.com/recetas/arroz-sofrito...,"['arroz', 'cebolla', 'pimiento rojo', 'panceta...",35.0,NaN,0,0,0,0,...,True,True,True,False,False,False,False,4,5,1.75


In [21]:
output_df.columns

Index(['recipe_image_url', 'recipe_name', 'recipe_url', 'ingredients_list',
       'time_preparation(min)', 'calories(kcal)', 'gluten_free', 'egg_free',
       'sucrose_fructose_free', 'low_sodium', 'lactose_free', 'tipo_plato',
       'main_category', 'category', 'num_ingredients', 'pechuga de pollo',
       'arroz', 'pimiento', 'aceite', 'sal', 'mantequilla', 'brécol',
       'calabacin', 'leche', 'active_ingred', 'num_missing_ingred',
       'recipe_rate'],
      dtype='object')

In [22]:
recipe_name = output_df["recipe_name"].iloc[0]

num_missing_ingred = output_df['num_missing_ingred'].iloc[0]

recipe_time = output_df["time_preparation(min)"].iloc[0]

print(f"La mejor receta que puedes hacer con los ingredientes que has introducido es: {recipe_name}")
print(f"Te faltan {num_missing_ingred} ingredientes")
print(f"El tiempo de preparación es de {recipe_time} minutos")

La mejor receta que puedes hacer con los ingredientes que has introducido es: Pechuga de pavo con chips de calabacín
Te faltan 3 ingredientes
El tiempo de preparación es de 40.0 minutos


In [23]:
go_to_recipe = input('¿Quieres ir a la receta?')

def normalize(string):
    lower = string.lower()
    a,b = 'áéíóú','aeiou'
    trans = str.maketrans(a,b)
    result = lower.translate(trans)
    return result

input_answer = normalize(go_to_recipe)

¿Quieres ir a la receta? si


In [24]:
if input_answer == 'si':
    recipe_link = output_df["recipe_url"].iloc[0]
    def pop_recipe(url_):
        ROUTE = url_
        webbrowser.open_new(ROUTE)
    pop_recipe(recipe_link)
elif input_answer == 'no':
    recipe_link = output_df["recipe_url"].iloc[0]
    print("Genial! Éste es el link por si quieres consultar la receta más tarde: {recipe_link}")
else:
    print("Por favor, responde sí o no")